In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [4]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [7]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [8]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [9]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [11]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [12]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [13]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [16]:
stream_brochure("X", "https://www.x.com")

Found links: {'relevant_links': ['https://www.x.com/about', 'https://www.x.com/services', 'https://www.x.com/careers', 'https://www.x.com/contact', 'https://www.x.com/testimonials', 'https://www.x.com/faq']}


# Welcome to X: Not Just a Letter of the Alphabet!

**Your one-stop destination for connecting, posting, and possibly losing track of time as you scroll through cat memes.**

---

## 🎉 Who Are We?
X is more than just a platform; it's a thriving online community where your friends become followers and your posts might just go viral (or play hide and seek in cyberspace). With us, you can exchange rapid-fire messages, share quirky posts, and find people with remarkably similar interests—or just follow your favorite celebrity's meal choices.

---

## 🚀 Our Services
At X, we pride ourselves on being a community-centric platform. Whether you're sharing moments, ideas, or the next great viral dance challenge, we've got you covered! Here’s what we do:

- **Interconnectivity**: Stay in touch with friends, family, coworkers, and that one guy you met at a party (because why not?).
- **User-Friendly Platform**: Easy peasy posting—post photos, videos, and text like a pro!
- **Explore**: Find content creators, businesses, and people who love the same weird stuff as you. P.S. We recommend plenty of stick figure art.

---

## 😄 Company Culture
Imagine a space where creativity and laughter are the norms. At X, we take our coffee seriously but not ourselves. Our office motto? "Less stress, more best-dressed." Work-life balance is our jam, and whether you’re brainstorming ideas or sharing office cats, we prioritize fun alongside productivity. 

---

## 💼 Career Opportunities
Looking to join a vibrant team where you can share cat pictures AND ideas? We’re hiring! At X, we offer:

- **Internships**: Kickstart your journey in a supportive environment.
- **Full-Time Positions**: Bring your wit, enthusiasm, and a sprinkle of craziness. We love that!
- **Developer Resources**: Don’t worry if you don’t know how to code. We have all sorts of tips and resources that would make the best code ninjas.

---

## 🙌 Our Customers
Our diverse users come from different walks of life, including:

- **Funny folks**: Masters of humor sharing their best dad jokes.
- **Creatives**: Photographers, writers, and digital artists exhibiting works that even your grandma might say, "that’s nice, dear."
- **Business Buffs**: Entrepreneurs who know that connecting on our platform might just land them their next big client or, at the very least, a dinner invite!

---

## 💬 FAQ
- **Q: What’s a post?**
  - A: It’s like a diary entry, but instead of being locked away, it’s broadcasted for everyone (including your boss).

- **Q: Can I edit my post?**
  - A: Editing is for X Premium subscribers only. For everyone else, sorry, once it’s out there, it’s out there for eternity!

- **Q: Can I follow people?**
  - A: Absolutely! It’s like social media stalking but with their consent!

---

Now that you've taken the tour, grab your X account and dive into the fun world of X! Join the conversation, share your thoughts, and maybe post a picture of your lunch like a true internet explorer.

---

## 🌐 Stay in Touch!
For more information, updates, or if you just want to say hi:
- [Instagram](https://www.instagram.com/x)
- [Twitter](https://www.twitter.com/x)
- [Facebook](https://www.facebook.com/x)

Remember, at X, we care about our users—because without you, we'd just be an "X" in the dictionary. Happy posting! 🎉